In [227]:
%pip install stepfunctions

import pandas as pd
import json
import boto3
import pathlib
import io
import sagemaker
from time import gmtime, strftime, sleep
from sagemaker.deserializers import CSVDeserializer
from sagemaker.serializers import CSVSerializer
import sagemaker
from sagemaker.experiments.run import Run, load_run
import time
from datetime import datetime
from sagemaker import image_uris
from sagemaker.estimator import Estimator

from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.processing import ProcessingInput, ProcessingOutput, ScriptProcessor
from sagemaker.inputs import TrainingInput

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import ProcessingStep, TrainingStep, CreateModelStep
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterFloat,
    ParameterString,
    ParameterBoolean,
)
from sagemaker import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.fail_step import FailStep

from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import Join, JsonGet

from sagemaker.drift_check_baselines import DriftCheckBaselines
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig
from sagemaker.image_uris import retrieve
from sagemaker.workflow.function_step import step
from sagemaker.workflow.step_outputs import get_step
from sagemaker.workflow.properties import PropertyFile

from sagemaker.workflow.conditions import (
    ConditionGreaterThan,
)

from stepfunctions.steps.sagemaker import EndpointConfigStep, EndpointStep

sagemaker.__version__

Note: you may need to restart the kernel to use updated packages.


'2.219.0'

In [228]:
# names of pipeline objects
project = "index-predictor"
bucket_name = "team1-index-predictor-bucket"
version = "v7"

current_timestamp = strftime("%d-%H-%M", gmtime())
pipeline_name = f"{project}-pipeline-{version}"
pipeline_model_name = f"{project}-model-xgb"
model_package_group_name = f"{project}-model-group-{version}"
endpoint_config_name = f"{project}-endpoint-config"
endpoint_name = f"{project}-endpoint"
model_approval_status = "PendingManualApproval"

# Set instance types and counts
process_instance_type = "ml.t3.medium"
process_instance_count = 1
train_instance_count = 1
model_endpoint_instance_count = 1
train_instance_type = "ml.m5.large"
model_endpoint_instance_type = "ml.t2.large"
endpoint_name = f"{project}-endpoint-{version}"

skprocessor_framework_version = "1.0-1"
session = PipelineSession()

# Thresholds
cumulative_return_threshold = -0.02
accuracy_threshold = 0.45

In [229]:
process_instance_type_param = ParameterString(
    name="ProcessingInstanceType",
    default_value=process_instance_type,
)
train_instance_type_param = ParameterString(
    name="TrainingInstanceType",
    default_value=train_instance_type,
)
train_instance_count_param = ParameterInteger(
    name="TrainingInstanceCount", default_value=train_instance_count
)
process_instance_count_param = ParameterInteger(
    name="ProcessingInstanceCount", default_value=process_instance_count
)
model_approval_status_param = ParameterString(
    name="ModelApprovalStatus", default_value=model_approval_status
)
version_param = ParameterString(
    name="Version", default_value=version
)

In [230]:
collection_processor = ScriptProcessor(
    command=["python3"],
    image_uri="492215442770.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-scikit-learn:1.0-1-cpu-py3",
    role=sagemaker.get_execution_role(),
    instance_count=process_instance_count,
    instance_type=process_instance_type,
)

collection_step = ProcessingStep(
    name=f"{project}-collection-{version}",
    processor=collection_processor,
    code="../src/data/collector.py",
    inputs=[],
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/output/data/raw",
            destination=f"s3://{bucket_name}/data/raw/{version}",
            output_name="raw_data",
        )
    ],
    job_arguments=[
        "--mode",
        "train-val-test",
        "--version",
        version,
    ],
)

In [231]:
processing_processor = ScriptProcessor(
    command=["python3"],
    image_uri="492215442770.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-scikit-learn:1.0-1-cpu-py3",
    role=sagemaker.get_execution_role(),
    instance_count=process_instance_count,
    instance_type=process_instance_type,
)

processing_step = ProcessingStep(
    name=f"{project}-processing-{version}",
    processor=processing_processor,
    code="../src/data/processor.py",
    inputs=[
        ProcessingInput(
            source=collection_step.properties.ProcessingOutputConfig.Outputs[
                "raw_data"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/input",
        )
    ],
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/output",
            destination=f"s3://{bucket_name}/data/processed/{version}",
            output_name="processed_data",
        )
    ],
    job_arguments=[
        "--raw_data_filename",
        f"/opt/ml/processing/input/data.csv",
        "--output_path",
        "/opt/ml/processing/output",
        "--version",
        version,
    ],
)

In [232]:
xgboost_container = image_uris.retrieve(
    "xgboost", session.boto_region_name, version="1.3-1"
)

estimator = Estimator(
    image_uri=xgboost_container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=f"s3://{bucket_name}/models/{version}",
    sagemaker_session=sagemaker.Session(),
    entry_point="../src/models/trainer.py",
)

estimator.set_hyperparameters(
    input_path="/opt/ml/input/data/train",
    data_version=version,
    target_column="Close_target",
    model_output_path="/opt/ml/model/model",
    num_trials=10,
)

train_input = TrainingInput(
    s3_data=processing_step.properties.ProcessingOutputConfig.Outputs[
        "processed_data"
    ].S3Output.S3Uri
)

training_step = TrainingStep(
    name=f"{project}-training-{version}",
    estimator=estimator,
    inputs={"train": train_input},
)

In [233]:
evaluation_processor = ScriptProcessor(
    command=["python3"],
    image_uri=xgboost_container,
    role=sagemaker.get_execution_role(),
    instance_count=process_instance_count,
    instance_type=process_instance_type,
)

evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation_result",
    path="evaluation_report.json",
)

evaluation_step = ProcessingStep(
    name=f"{project}-evaluation-{version}",
    processor=evaluation_processor,
    code="../src/models/evaluator.py",
    inputs=[
        ProcessingInput(
            source=processing_step.properties.ProcessingOutputConfig.Outputs[
                "processed_data"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/input",
        ),
        ProcessingInput(
            source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
    ],
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/output",
            destination=f"s3://{bucket_name}/evaluation_results/{version}",
            output_name="evaluation_result",
        )
    ],
    job_arguments=[
        "--input_path",
        "/opt/ml/processing/input",
        "--data_version",
        version,
        "--target_column",
        "Close_target",
        "--model_path",
        "/opt/ml/processing/model/model",
        "--output_path",
        "/opt/ml/processing/output",
    ],
    property_files=[evaluation_report],
)

In [234]:
model = Model(
    image_uri=xgboost_container,
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    role=sagemaker.get_execution_role(),
    sagemaker_session=session,
)

In [235]:
step_fail = FailStep(
    name=f"{project}-fail-{version}",
    error_message=f"Execution failed due to Cumulative Return below {cumulative_return_threshold} or Accuracy below {accuracy_threshold}",
)

In [236]:
cond_cumulative_return = ConditionGreaterThan(
    left=JsonGet(
        step=evaluation_step,
        property_file=evaluation_report,
        json_path="Cumulative Return",
    ),
    right=cumulative_return_threshold,
)

cond_accuracy = ConditionGreaterThan(
    left=JsonGet(
        step=evaluation_step,
        property_file=evaluation_report,
        json_path="Test Accuracy",
    ),
    right=accuracy_threshold,
)

check_model_step = ConditionStep(
    name=f"{project}-check-model-{version}",
    conditions=[cond_cumulative_return, cond_accuracy],
    if_steps=[],
    else_steps=[step_fail],
)

In [237]:
pipeline_def_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        process_instance_type_param,
        process_instance_count_param,
        train_instance_type_param,
        train_instance_count_param,
        model_approval_status_param,
        version_param,
    ],
    steps=[collection_step, processing_step, training_step, evaluation_step],
    sagemaker_session=session,
    pipeline_definition_config=pipeline_def_config,
)

pipeline.upsert(role_arn=sagemaker.get_execution_role())

{'PipelineArn': 'arn:aws:sagemaker:eu-central-1:567821811420:pipeline/index-predictor-pipeline-v7',
 'ResponseMetadata': {'RequestId': 'd636682b-936f-4d07-8b7a-bfa1db4ab671',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd636682b-936f-4d07-8b7a-bfa1db4ab671',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '98',
   'date': 'Wed, 19 Jun 2024 22:49:07 GMT'},
  'RetryAttempts': 0}}

In [238]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType=process_instance_type,
        TrainingInstanceType=train_instance_type,
        TrainingInstanceCount=train_instance_count,
        ModelApprovalStatus="PendingManualApproval",
    )
)